In [3]:
import pandas as pd
import numpy as np
import random
from sklearn import linear_model

In [4]:
#database=pd.read_pickle('311_time_difference')
#database.head()
database=pd.read_csv('311_time_difference')
database.head()

C:\Users\Marco\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Resolution Description,Created_TO,Closed_TO,delta,difference
0,0,33022454,03/31/2016 02:10:00 AM,03/31/2016 02:30:31 AM,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11204,The Police Department responded to the complai...,2016-03-31 02:10:00,2016-03-31 02:30:31,0 days 00:20:31.000000000,20.516667
1,1,33023558,03/31/2016 01:48:58 AM,03/31/2016 03:02:20 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11367,The Police Department responded and upon arriv...,2016-03-31 01:48:58,2016-03-31 03:02:20,0 days 01:13:22.000000000,73.366667
2,2,33024431,03/31/2016 01:43:32 AM,03/31/2016 01:55:13 AM,NYPD,New York City Police Department,Noise - Commercial,Banging/Pounding,Club/Bar/Restaurant,10009,The Police Department responded to the complai...,2016-03-31 01:43:32,2016-03-31 01:55:13,0 days 00:11:41.000000000,11.683333
3,3,33024039,03/31/2016 01:42:59 AM,03/31/2016 02:45:19 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11218,The Police Department issued a summons in resp...,2016-03-31 01:42:59,2016-03-31 02:45:19,0 days 01:02:20.000000000,62.333333
4,4,33024694,03/31/2016 01:37:16 AM,03/31/2016 02:10:11 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11365,The Police Department issued a summons in resp...,2016-03-31 01:37:16,2016-03-31 02:10:11,0 days 00:32:55.000000000,32.916667


In [5]:
#Since originally the data doesn't contain borough or time difference, I'm going to simmulate those attributes
#random.seed(42)
#database['difference']=np.random.choice(range(1, 1000), database.shape[0])
#database['borough']=np.random.choice(range(1,6), database.shape[0])

In [6]:
cols=[u'Agency',
     u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip',u'difference']

In [7]:
database=database[cols]
database.head()

,Agency,Complaint Type,Descriptor,Location Type,Incident Zip,difference
0,NYPD,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,11204,20.516667
1,NYPD,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11367,73.366667
2,NYPD,Noise - Commercial,Banging/Pounding,Club/Bar/Restaurant,10009,11.683333
3,NYPD,Blocked Driveway,No Access,Street/Sidewalk,11218,62.333333
4,NYPD,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11365,32.916667


In [8]:
noise311_cut=database.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [9]:
#X = noise311_cut[['Agency','Complaint Type','Descriptor','Location Type','Incident Zip','borough']]
#X=noise311_cut[['Agency','Complaint Type','Descriptor','Location Type','borough']]
X=noise311_cut[['Agency','Complaint Type','Descriptor','Location Type','Incident Zip']]

In [10]:
Y=noise311_cut[['difference']]
Y=np.array(Y)
Y=[item for sublist in Y for item in sublist]

In [ ]:
X_dum= pd.get_dummies(X)

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_dum,Y,test_size=0.33,random_state=990) 

In [ ]:
Lasso=linear_model.Lasso(fit_intercept=True,alpha=10) #try Ridge with a selected regularization parameter lambda

Lasso.fit(X_train,y_train)
# In the sample:
p_IS=Lasso.predict(X_train)
err_IS=p_IS-y_train
R_2_IS_Lasso=1-np.var(err_IS)/np.var(y_train)
print("The R-squared we found for IS Lasso is: {0}".format(R_2_IS_Lasso))

Lasso_coef=Lasso.coef_
############################################################################    
    
#Out of sample
p_OS=Lasso.predict(X_test)
err_OS=p_OS-y_test
R_2_OS_Lasso=1-np.var(err_OS)/np.var(y_test)
print("The R-squared we found for OS Lasso is: {0}".format(R_2_OS_Lasso))